In [1]:
import pandas as pd
#load merged.csv
merged = pd.read_csv(r'C:\Users\zuzan\OneDrive\Desktop\Python Load\Buurt_.csv', sep = ',')
merged.buurtcode = merged.buurtcode.astype(str)
merged = merged[['buurtcode', 'buurtname']]
merged.drop_duplicates(subset ='buurtcode', keep = 'first', inplace = True)
print(merged.shape)
print(merged.head())

(13305, 2)
  buurtcode                                    buurtname
0     30000                           Appingedam-Centrum
1     30001                              Appingedam-West
2     30002                              Appingedam-Oost
3     30007  Verspreide huizen Damsterdiep en Eemskanaal
4     30008  Verspreide huizen ten zuiden van Eemskanaal


In [3]:
import pandas as pd
income = pd.read_csv(r'C:\Users\zuzan\OneDrive\Desktop\repository\product3team9\data\raw\cbs_datasets\cbs_income_social_security.csv', 
                     error_bad_lines = False, sep = ';')
income = income[['NBH_code', '%_low_income_households','%_below_social_minimum']]
income.head()

,NBH_code,%_low_income_households,%_below_social_minimum
0,NL00,7.9,7.1
1,GM1680,5.4,4.9
2,WK168000,3.5,4.5
3,BU16800000,3.4,4.5
4,BU16800009,.,.


In [4]:
income = income[income['NBH_code'].str.contains('BU')]
income['NBH_code'] = income['NBH_code'].astype(str)
for a in range(len(income)):
    if income['NBH_code'].iloc[a][2:3] == '0' and income['NBH_code'].iloc[a][3:4] != '0':
        income['NBH_code'].iloc[a] = ''.join(income['NBH_code'].iloc[a].rsplit('BU0'))
    elif income['NBH_code'].iloc[a][2:4] == '00' and income['NBH_code'].iloc[a][4:5] != '0':
        income['NBH_code'].iloc[a] = ''.join(income['NBH_code'].iloc[a].rsplit('BU00'))
    elif income['NBH_code'].iloc[a][2:5] == '000':
        income['NBH_code'].iloc[a] = ''.join(income['NBH_code'].iloc[a].rsplit('BU000'))
    else:
        income['NBH_code'].iloc[a] = income['NBH_code'].iloc[a][2:]
   
income.head(100)

,NBH_code,%_low_income_households,%_below_social_minimum
3,16800000,3.4,4.5
4,16800009,.,.
6,16800100,6.9,6.1
7,16800109,.,.
9,16800200,7.6,6.9
...,...,...,...
134,1970302,3.4,2.8
135,1970303,2.4,1.8
136,1970304,2.7,2.9
137,1970305,2.2,1.5


In [6]:
cleaned_income = pd.merge(income, merged, left_on='NBH_code', right_on='buurtcode', how = 'left')
cleaned_income = cleaned_income.loc[cleaned_income['buurtname'].isna() == False]
cleaned_income = cleaned_income[['buurtname','%_low_income_households','%_below_social_minimum']]
cleaned_income.shape

(13305, 3)

In [12]:
cleaned_income.isna().sum()

buurtname                     0
%_low_income_households    4193
%_below_social_minimum     4193
dtype: int64

In [10]:
cleaned_income['%_low_income_households']=cleaned_income['%_low_income_households'].str.extract(r'(\d)')
cleaned_income['%_below_social_minimum']=cleaned_income['%_below_social_minimum'].str.extract(r'(\d)')

In [ ]:
cleaned_income = cleaned_income.dropna(subset = ['%_low_income_households', '%_below_social_minimum'])

In [16]:
cleaned_income['%_low_income_households']=cleaned_income['%_low_income_households'].astype(int)
cleaned_income['%_below_social_minimum']=cleaned_income['%_below_social_minimum'].astype(int)
print(cleaned_income.dtypes)

buurtname                  object
%_low_income_households     int32
%_below_social_minimum      int32
dtype: object


<ipython-input-16-9f1e311e8108>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_income['%_low_income_households']=cleaned_income['%_low_income_households'].astype(int)
<ipython-input-16-9f1e311e8108>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_income['%_below_social_minimum']=cleaned_income['%_below_social_minimum'].astype(int)


In [17]:
print(cleaned_income.dtypes)

buurtname                  object
%_low_income_households     int32
%_below_social_minimum      int32
dtype: object


In [19]:
cleaned_income=cleaned_income.drop_duplicates(subset='buurtname', keep='first', inplace=False, ignore_index=False)

In [20]:
cleaned_income.shape

(8449, 3)

In [22]:
cleaned_income.to_csv('cleaned_income.csv', index=False)